# Homework Assignment  6

**Question 1:** Construct a sentiment classifier using 80% of the reviews in the [Amazon Fine Food Reviews dataset](https://www.kaggle.com/snap/amazon-fine-food-reviews).
The classifier needs to predict if a product got a one-star or a five-star review. 
Evaluate the classifier using the rest of the data (30pt). 


In [ ]:
!pip install turicreate
!pip install afinn
!pip install TextBlob
!pip install spaCy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import turicreate as tc

In [ ]:
import json
!mkdir /root/.kaggle/
# Installing the Kaggle package
!pip install kaggle 

#!!!fill your api key!!!
api_token = {"username":"","key":""}

with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/amazon-food-reviews

# download the dataset from Kaggle and unzip it
!kaggle datasets download snap/amazon-fine-food-reviews  -f Reviews.csv -p ./datasets/amazon-food-reviews/
!unzip ./datasets/amazon-food-reviews/*.zip  -d ./datasets/amazon-food-reviews
!ls ./datasets/amazon-food-reviews

mkdir: cannot create directory ‘/root/.kaggle/’: File exists
mkdir: cannot create directory ‘./datasets’: File exists
mkdir: cannot create directory ‘./datasets/amazon-food-reviews’: File exists
Reviews.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/amazon-food-reviews/Reviews.csv.zip
replace ./datasets/amazon-food-reviews/Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./datasets/amazon-food-reviews/Reviews.csv  
Reviews.csv  Reviews.csv.zip


In [ ]:
import pandas as pd

# Loading the data to dataframe first because tc is having problems reading the csv
reviews_df = pd.read_csv("./datasets/amazon-food-reviews/Reviews.csv")#, nrows=10000)
reviews_df = reviews_df.dropna()
reviews_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [ ]:
reviews_sf = tc.SFrame(reviews_df)
reviews_sf.materialize()
reviews_sf = reviews_sf["Score", "Summary", "Text"]

reviews_sf['full_text'] = reviews_sf.apply(lambda r: str(r['Summary']+" "+r['Text']).lower())
reviews_sf = reviews_sf["Score", "full_text"]
reviews_sf

Score,full_text
5,good quality dog food ihave bought several of ...
1,not as advertised productarrived labeled as jumbo ...
4,"""delight"" says it allthis is a confection ..."
2,cough medicine if you arelooking for the secret ...
5,great taffy great taffyat a great price. there ...
4,nice taffy i got a wildhair for taffy and ...
5,great! just as good asthe expensive brands! ...
5,"wonderful, tasty taffythis taffy is so good. ..."
5,yay barley right now i'mmostly just sprouting ...
5,healthy dog food this isa very healthy dog food. ...


In [ ]:
import seaborn as sns
import plotly.express as px

sns.set()
sns.set_style("whitegrid")
# sns.distplot(reviews_sf['Score'], kde=False, rug=True,color="c")
fig = px.histogram(reviews_df, x="Score", color="Score")
fig.show()
reviews_sf.groupby('Score', tc.aggregate.COUNT()).sort("Count", ascending=False).print_rows(5)

+-------+--------+
| Score | Count  |
+-------+--------+
|   5   | 363111 |
|   4   | 80655  |
|   1   | 52264  |
|   3   | 42638  |
|   2   | 29743  |
+-------+--------+
[5 rows x 2 columns]



In [ ]:
# The data is heavily unbalanced.
# To somewhat mitigate that and to lose less data 
# we will regard the reviews with 2 stars as reviews with a single star.
# Reviews with 4 and 3 stars ratings are dropped
reviews_sf = reviews_sf[reviews_sf.apply(lambda r: r['Score'] != 3 and r['Score'] != 4)]
reviews_sf['Score'] = reviews_sf['Score'].apply(lambda t: t if t!=2 else 1)
reviews_sf.materialize()
reviews_sf['words_1grams'] = tc.text_analytics.count_ngrams(reviews_sf['full_text'], n=1, method='word')
reviews_sf['words_1grams'] = reviews_sf['words_1grams'].dict_trim_by_keys(tc.text_analytics.stop_words(), True)
reviews_sf['words_2grams'] = tc.text_analytics.count_ngrams(reviews_sf['full_text'], n=2, method='word')
reviews_sf

Score,full_text,words_1grams,words_2grams
5,good quality dog food ihave bought several of ...,"{'appreciates': 1,'labrador': 1, 'smells': ...","{'product better': 1, 'ofgood': 1, 'stew than' ..."
1,not as advertised productarrived labeled as jumbo ...,"{'intended': 1, 'vendor':1, 'arrived': 1, ...","{'the product': 1,'represent the': 1, 'to ..."
1,cough medicine if you arelooking for the secret ...,"{'medicinal': 1,'flavor': 1, 'soda': 1, ...","{'is very': 1, 'sodathe': 1, 'cherry soda': ..."
5,great taffy great taffyat a great price. there ...,"{'deal': 1, 'lover': 1,'quick': 1, 'delivery': ...","{'lover this': 1, 'taffylover': 1, 'a wide': 1, ..."
5,great! just as good asthe expensive brands! ...,"{'loved': 1, 'chewy': 1,'wrapped': 1, 'party' ...","{'loved it': 1, 'everyoneloved': 1, 'and ..."
5,"wonderful, tasty taffythis taffy is so good. ...","{'satisfying': 1, 'soft':1, 'buying': 1, 'good': ...","{'it very': 1, 'good it':1, 'taffy is': 1, 'the ..."
5,yay barley right now i'mmostly just sprouting ...,"{'rye': 1, 'eat': 1,'wheatgrass': 1, ...","{'rye too': 1, 'and rye':1, 'with wheatgrass': 1, ..."
5,healthy dog food this isa very healthy dog food. ...,"{'feeding': 1,'required': 1, 'eats' ...","{'at every': 1, 'foodthis': 1, 'her requir ..."
5,the best hot sauce in theworld i don't know if ...,"{'service': 1,'incredible': 1, ...","{'personal incredible':1, 'you for': 1, 'thank ..."
5,"my cats love this ""diet""food better than their ...","{'week': 1, 'skinny': 1,'weight': 1, 'lose': 1, ...","{'a week': 1, 'an ounce':1, 'losing about': 1, ..."


In [ ]:
# train test split
train, test = reviews_sf.random_split(0.8)
cls = tc.logistic_classifier.create(train,features=['words_1grams', 'words_2grams'], target="Score")

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 338658

Number of classes           : 2

...

| 6         | 14       | 1.896773  | 33.252686    | 0.999897          | 0.969425            |

| 7         | 16       | 1.000000  | 38.106177    | 0.999956          | 0.969257            |

| 8         | 18       | 1.000000  | 42.952982    | 0.999965          | 0.969481            |

| 9         | 19       | 1.000000  | 45.954947    | 0.999962          | 0.969593            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [ ]:
results = cls.evaluate(test)
results

{'accuracy': 0.9704744175551419,
 'auc': 0.9872314589836602,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      5       |        1        |  763  |
 |      1       |        5        |  1854 |
 |      1       |        1        | 14506 |
 |      5       |        5        | 71512 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9820311588083026,
 'log_loss': 0.09795487559156843,
 'precision': 0.9747294387045771,
 'recall': 0.9894430992736077,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+---------------------+--------------------+-------+-------+
 | threshold |         fpr         |        tpr         |   p   |   n   |
 +-----------+---------------------+--------------------+-------+-------+
 | 

In [ ]:
fig = px.line(x=results['roc_curve']['fpr'], y=results['roc_curve']['tpr'], title='ROC Curve (AUC = %0.5f)' %results['auc'])
fig.show()

**Question 2:** Construct a sentiment classifier using features from the [TextBlob package](https://textblob.readthedocs.io/en/dev/index.html),
    and from the [affin package](https://pypi.org/project/afinn/). Compare the two classifiers results and visualize it (however you want) (20pt)

In [ ]:
from afinn import Afinn
afinn = Afinn()
train['afinn_score'] = train['full_text'].apply(lambda t: afinn.score(t))
test['afinn_score'] = test['full_text'].apply(lambda t: afinn.score(t))
afinn_cls = tc.logistic_classifier.create(train,features=['afinn_score', 'words_1grams', 'words_2grams'], target="Score")
afinn_results = afinn_cls.evaluate(test)
afinn_results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 338658

Number of classes           : 2

...

| 3         | 11       | 1.910370  | 25.457923    | 0.999091          | 0.964544            |

| 4         | 12       | 1.910370  | 28.525308    | 0.999663          | 0.968359            |

| 6         | 14       | 1.910370  | 34.659322    | 0.999870          | 0.969369            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

{'accuracy': 0.9699554352118238,
 'auc': 0.9885204741297278,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      5       |        1        |  672  |
 |      1       |        5        |  1991 |
 |      1       |        1        | 14369 |
 |      5       |        5        | 71603 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.981743893493477,
 'log_loss': 0.09128066824101361,
 'precision': 0.9729461640894638,
 'recall': 0.9907021791767554,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+---------------------+--------------------+-------+-------+
 | threshold |         fpr         |        tpr         |   p   |   n   |
 +-----------+---------------------+--------------------+-------+-------+
 |  

In [ ]:
from textblob import TextBlob
train['text_blob_polarity'] = train['full_text'].apply(lambda t: TextBlob(t).sentiment.polarity)
test['text_blob_polarity'] = test['full_text'].apply(lambda t: TextBlob(t).sentiment.polarity)
train['text_blob_subjectivity'] = train['full_text'].apply(lambda t: TextBlob(t).sentiment.subjectivity)
test['text_blob_subjectivity'] = test['full_text'].apply(lambda t: TextBlob(t).sentiment.subjectivity)
textblob_cls = tc.logistic_classifier.create(train,features=['text_blob_polarity','text_blob_subjectivity', 'words_1grams', 'words_2grams'], target="Score")
textblob_results = textblob_cls.evaluate(test)
textblob_results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 338658

Number of classes           : 2

...

| 6         | 15       | 1.396013  | 38.684269    | 0.999917          | 0.969313            |

| 7         | 16       | 1.396013  | 41.946400    | 0.999938          | 0.970042            |

| 8         | 17       | 1.396013  | 45.113208    | 0.999938          | 0.970042            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

{'accuracy': 0.9703503130817397,
 'auc': 0.9897107497426892,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      5       |        1        |  729  |
 |      1       |        5        |  1899 |
 |      1       |        1        | 14461 |
 |      5       |        5        | 71546 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.981965413121054,
 'log_loss': 0.09045003832216786,
 'precision': 0.974143917216965,
 'recall': 0.9899135247319266,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+---------------------+--------------------+-------+-------+
 | threshold |         fpr         |        tpr         |   p   |   n   |
 +-----------+---------------------+--------------------+-------+-------+
 |   

In [ ]:
fig = px.line(x=afinn_results['roc_curve']['fpr'], y=afinn_results['roc_curve']['tpr'], title='Affin ROC Curve (AUC = %0.5f)' %afinn_results['auc'])
fig.show()

In [ ]:
fig = px.line(x=textblob_results['roc_curve']['fpr'], y=textblob_results['roc_curve']['tpr'], title='TextBlob ROC Curve (AUC = %0.5f)' %textblob_results['auc'])
fig.show()

The better model is the one with the higher AUC

**Question 3:**  Select a different reviews dataset and create a sentiment classifier which utilizes word embeddings (25pt).
Evaluate this classifier (5pt). Try to improve your classifier by adding additional features (20pt)

In [ ]:
!mkdir ./datasets/clothing-reviews

# download the dataset from Kaggle and unzip it
!kaggle datasets download nicapotato/womens-ecommerce-clothing-reviews -p ./datasets/clothing-reviews/
!unzip ./datasets/clothing-reviews/*.zip  -d ./datasets/clothing-reviews
!ls ./datasets/clothing-reviews

mkdir: cannot create directory ‘./datasets/clothing-reviews’: File exists
womens-ecommerce-clothing-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/clothing-reviews/womens-ecommerce-clothing-reviews.zip
replace ./datasets/clothing-reviews/Womens Clothing E-Commerce Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./datasets/clothing-reviews/Womens Clothing E-Commerce Reviews.csv  
'Womens Clothing E-Commerce Reviews.csv'
 womens-ecommerce-clothing-reviews.zip


In [ ]:
clothing_reviews_sf = tc.SFrame.read_csv("./datasets/clothing-reviews/Womens Clothing E-Commerce Reviews.csv")
clothing_reviews_sf.materialize()
clothing_reviews_sf = clothing_reviews_sf["Title", "Review Text", "Rating"]

clothing_reviews_sf['full_text'] = clothing_reviews_sf.apply(lambda r: str(r['Title']+" "+r['Review Text']).lower())
clothing_reviews_sf = clothing_reviews_sf["Rating", "full_text"]
clothing_reviews_sf

Finished parsing file /content/datasets/clothing-reviews/Womens Clothing E-Commerce Reviews.csv

Parsing completed. Parsed 100 lines in 0.267377 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,str,str,int,int,int,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/datasets/clothing-reviews/Womens Clothing E-Commerce Reviews.csv

Parsing completed. Parsed 23486 lines in 0.198238 secs.

Rating,full_text
4,absolutely wonderful -silky and sexy and ...
5,love this dress! it'ssooo pretty. i happened ...
3,some major design flaws ihad such high hopes for ...
5,"my favorite buy! i love,love, love this jumps ..."
5,flattering shirt thisshirt is very flattering ...
2,not for the very petite ilove tracy reese dres ...
5,cagrcoal shimmer fun iaded this in my baske ...
4,"shimmer, surprisinglygoes with lots i ordered ..."
5,flattering i love thisdress. i usually get an ...
5,"such a fun dress! i'm5""5' and 125 lbs. i ..."


In [ ]:
sns.set()
sns.set_style("whitegrid")
# sns.distplot(reviews_sf['Score'], kde=False, rug=True,color="c")
fig = px.histogram(clothing_reviews_sf.to_dataframe(), x="Rating", color="Rating")
fig.show()
clothing_reviews_sf.groupby('Rating', tc.aggregate.COUNT()).sort("Count", ascending=False).print_rows(5)

+--------+-------+
| Rating | Count |
+--------+-------+
|   5    | 13131 |
|   4    |  5077 |
|   3    |  2871 |
|   2    |  1565 |
|   1    |  842  |
+--------+-------+
[5 rows x 2 columns]



In [ ]:
# The data is heavily unbalanced.
# To somewhat mitigate that and to lose less data 
# we will regard the reviews with 2 stars as reviews with a single star.
# Reviews with 4 and 3 stars ratings are dropped
clothing_reviews_sf = clothing_reviews_sf[clothing_reviews_sf.apply(lambda r: r['Rating'] != 3)]
clothing_reviews_sf['Rating'] = clothing_reviews_sf['Rating'].apply(lambda t: 1 if t<3 else 5)
clothing_reviews_sf.materialize()
clothing_reviews_sf['words_1grams'] = tc.text_analytics.count_ngrams(clothing_reviews_sf['full_text'], n=1, method='word')
clothing_reviews_sf['words_1grams'] = clothing_reviews_sf['words_1grams'].dict_trim_by_keys(tc.text_analytics.stop_words(), True)
clothing_reviews_sf['words_2grams'] = tc.text_analytics.count_ngrams(clothing_reviews_sf['full_text'], n=2, method='word')
clothing_reviews_sf['afinn_score'] = clothing_reviews_sf['full_text'].apply(lambda t: afinn.score(t))
clothing_reviews_sf['text_blob_polarity'] = clothing_reviews_sf['full_text'].apply(lambda t: TextBlob(t).sentiment.polarity)
clothing_reviews_sf['text_blob_subjectivity'] = clothing_reviews_sf['full_text'].apply(lambda t: TextBlob(t).sentiment.subjectivity)

clothing_reviews_sf

Rating,full_text,words_1grams,words_2grams,afinn_score
5,absolutely wonderful -silky and sexy and ...,"{'comfortable': 1,'sexy': 1, 'silky': 1, ...","{'sexy and': 1, 'silkyand': 1, 'wonderful ...",9.0
5,love this dress! it'ssooo pretty. i happened ...,"{'true': 1, 'hits': 1,'8': 1, '5': 1, 'peti ...","{'who is': 1, 'onsomeone': 1, 'true mi ...",12.0
5,"my favorite buy! i love,love, love this jumps ...","{'compliments': 1,'time': 1, 'fabulous' ...","{'great compliments': 1,'but great': 1, 's fun': ...",24.0
5,flattering shirt thisshirt is very flattering ...,"{'sleeveless': 1,'cardigan': 1, 'due': 1, ...","{'cardigan love': 1,'pairs well': 1, 'love ...",6.0
1,not for the very petite ilove tracy reese dres ...,"{'returned': 1, 'work':1, 'style': 1, 'idea' ...","{'i returned': 1, 'onme': 1, 'not work': 1, ...",7.0
5,cagrcoal shimmer fun iaded this in my baske ...,"{'3': 1, 'pants': 1,'jkeep': 1, 'bummer': 1, ...","{'kept all': 1, 'whichi': 1, 'turns out': 1 ...",1.0
5,"shimmer, surprisinglygoes with lots i ordered ...","{'smallest': 1, 'light':1, 'ldecide': 1, 'hte': ...","{'hte smallest': 1, 'inhte': 1, 'this in': 1, ...",2.0
5,flattering i love thisdress. i usually get an ...,"{'style': 1, 'flair': 1,'retailer': 1, 'usual': ...","{'retailer flair': 1,'the usual': 1, 'and ...",3.0
5,"such a fun dress! i'm5""5' and 125 lbs. i ...","{'longer': 1, 'love': 2,'long': 1, 'underneath': ...","{'fabric underneath': 1,'longer then': 1, 'the ...",14.0
5,this dress isperfection! so pretty ...,"{'flattering': 1,'pretty': 1, 'dress': 1, ...","{'and flattering': 1,'pretty and': 1, 'so ...",4.0


In [ ]:
from tqdm import tqdm
import cupy as cp
import numpy as np
import spacy
spacy.require_gpu()
nlp = spacy.load('en_core_web_lg')

l = []
for i in tqdm(range(len(clothing_reviews_sf))):
    text = clothing_reviews_sf[i]["full_text"]
    l.append(cp.asnumpy(nlp(text).vector))
clothing_reviews_sf['embedding'] = l 
clothing_reviews_sf

100%|██████████| 20615/20615 [15:07<00:00, 22.73it/s]


Rating,full_text,words_1grams,words_2grams,afinn_score
5,absolutely wonderful -silky and sexy and ...,"{'comfortable': 1,'sexy': 1, 'silky': 1, ...","{'sexy and': 1, 'silkyand': 1, 'wonderful ...",9.0
5,love this dress! it'ssooo pretty. i happened ...,"{'true': 1, 'hits': 1,'8': 1, '5': 1, 'peti ...","{'who is': 1, 'onsomeone': 1, 'true mi ...",12.0
5,"my favorite buy! i love,love, love this jumps ...","{'compliments': 1,'time': 1, 'fabulous' ...","{'great compliments': 1,'but great': 1, 's fun': ...",24.0
5,flattering shirt thisshirt is very flattering ...,"{'sleeveless': 1,'cardigan': 1, 'due': 1, ...","{'cardigan love': 1,'pairs well': 1, 'love ...",6.0
1,not for the very petite ilove tracy reese dres ...,"{'returned': 1, 'work':1, 'style': 1, 'idea' ...","{'i returned': 1, 'onme': 1, 'not work': 1, ...",7.0
5,cagrcoal shimmer fun iaded this in my baske ...,"{'3': 1, 'pants': 1,'jkeep': 1, 'bummer': 1, ...","{'kept all': 1, 'whichi': 1, 'turns out': 1 ...",1.0
5,"shimmer, surprisinglygoes with lots i ordered ...","{'smallest': 1, 'light':1, 'ldecide': 1, 'hte': ...","{'hte smallest': 1, 'inhte': 1, 'this in': 1, ...",2.0
5,flattering i love thisdress. i usually get an ...,"{'style': 1, 'flair': 1,'retailer': 1, 'usual': ...","{'retailer flair': 1,'the usual': 1, 'and ...",3.0
5,"such a fun dress! i'm5""5' and 125 lbs. i ...","{'longer': 1, 'love': 2,'long': 1, 'underneath': ...","{'fabric underneath': 1,'longer then': 1, 'the ...",14.0
5,this dress isperfection! so pretty ...,"{'flattering': 1,'pretty': 1, 'dress': 1, ...","{'and flattering': 1,'pretty and': 1, 'so ...",4.0


In [ ]:
train, test  = clothing_reviews_sf.random_split(0.8)
clothing_reviews_cls_embeds_only = tc.logistic_classifier.create(train,features=['embedding'], target="Rating")
results_embeds_only = clothing_reviews_cls_embeds_only.evaluate(test)
results_embeds_only

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 15629

Number of classes           : 2

...

| 7         | 8        | 24.150836    | 0.947853          | 0.921021            |

| 8         | 9        | 27.173605    | 0.947853          | 0.921021            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

{'accuracy': 0.9363439827047803,
 'auc': 0.9470196530486142,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      5       |        1        |   83  |
 |      1       |        5        |  182  |
 |      1       |        1        |  310  |
 |      5       |        5        |  3588 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9643865071898938,
 'log_loss': 0.17554267236101295,
 'precision': 0.9517241379310345,
 'recall': 0.9773903568509943,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+------+-----+
 | threshold |        fpr         |        tpr         |  p   |  n  |
 +-----------+--------------------+--------------------+------+-----+
 |    0.0    | 

In [ ]:
clothing_reviews_cls_all_featurs = tc.logistic_classifier.create(train,features=['embedding','text_blob_subjectivity','text_blob_polarity', 'afinn_score'], target="Rating")
results_all_featurs = clothing_reviews_cls_all_featurs.evaluate(test)
results_all_featurs

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 15629

Number of classes           : 2

...

| 7         | 8        | 24.033505    | 0.948493          | 0.939247            |

| 8         | 9        | 27.104601    | 0.948493          | 0.939247            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

{'accuracy': 0.9363439827047803,
 'auc': 0.9473961482327983,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      5       |        1        |   86  |
 |      1       |        5        |  179  |
 |      1       |        1        |  313  |
 |      5       |        5        |  3585 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9643577673167452,
 'log_loss': 0.1743401019502371,
 'precision': 0.9524442082890542,
 'recall': 0.9765731408335603,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+------+-----+
 | threshold |        fpr         |        tpr         |  p   |  n  |
 +-----------+--------------------+--------------------+------+-----+
 |    0.0    |  

In [ ]:
fig = px.line(x=results_embeds_only['roc_curve']['fpr'], y=results_embeds_only['roc_curve']['tpr'], title='embeds_only ROC Curve (AUC = %0.5f)' %results_embeds_only['auc'])
fig.show()

In [ ]:
fig = px.line(x=results_all_featurs['roc_curve']['fpr'], y=results_all_featurs['roc_curve']['tpr'], title='all_featurs ROC Curve (AUC = %0.5f)' %results_all_featurs['auc'])
fig.show()